# Overview

This tutorial is about loading an Ollama model and interacting with it. This interaction includes messages. The tutorial ends by makeing a simple chat application that keeps chat history.

# Load a chat model

Chat models can generate text by continuing a given prompt. First we need to load a model that has been pulled via Ollama (see README.md for details).

In [1]:
from langchain_ollama import ChatOllama

In [2]:
# Initialize the ChatOllama model with the specified model name
model_name = 'qwen3:4b'

# and initialize the ChatOllama instance
chat_model = ChatOllama(
    model=model_name,
    validate_model_on_init=True,
    temperature=0.7
)

## Model invocation

We can invoke a model, i.e., give it a prompt to generate output, either by passing a string of the prompt, or by providing messages. Messages are either in the form of dictionaries or special LangChain classes

### Invocation with direct text

We can invoke the message with a simple string that includes the prompt.

In [3]:
# Invoke model with string input
response = chat_model.invoke("How can I make a delicious chocolate cake?")

In [4]:
# The response is an AIMessage object that includes the model's reply
print(type(response))

# We can get the content of the response like this
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
Here's a **simple, reliable, and delicious chocolate cake recipe** that beginners and experts alike love (tested for perfect moisture, flavor, and texture!). I'll focus on **why each step matters** to ensure your cake turns out **rich, moist, and deeply chocolatey**—no dryness, cracks, or blandness.

---

### 🎯 Why This Recipe Works (Key Secrets to "Delicious")
1. **Dark chocolate > milk chocolate** (for intense flavor without sugar overload).
2. **Sifting flour + cocoa powder** = air pockets = tender crumb (no dense cake!).
3. **Butter + oil blend** = extra moisture (prevents dryness).
4. **Baking at 350°F (175°C)** = even rise (too hot = tough cake).
5. **Don’t overmix** = keeps cake light (overmixing = tough, heavy texture).

---

### 📝 Your Perfect Chocolate Cake Recipe (Makes 10–12 servings)
#### 🧾 Ingredients (All in metric for precision)
| **Ingredient**          | **Amount**          | **Why It Matters**                            

### Invocation with LangChain messages

We can invoke a message with LangChain messages. These messages are objects of classes, like the `AIMessage`, that are intepreted by models as having different roles within the chat, affecting how the model parses them differently.

Along with the `AIMessage` class, there is a `SystemMessage` and a `HumanMessage class`. There is also a `TollMessage` class, but we will see it in a next tutorial.

- `SystemMessage` - Tells the model how to behave and provide context for interactions.
- `HumanMessage` - Represents user input and interactions with the model.
- `AIMessage` - Responses generated by the model, including text content, tool calls, and metadata.
- `ToolMessge` - Represents the outputs of tool calls (next tutorial).

For more information on messages you can visit: https://docs.langchain.com/oss/python/langchain/messages#tool-message

In [5]:
# Let's initialize a system message and a human message
from langchain.messages import SystemMessage, HumanMessage
system_message = SystemMessage(content="You are a helpful assistant that provides cooking tips.")
human_message = HumanMessage(content="Can you suggest a recipe for chocolate cake?")

messages = [system_message, human_message]

# Now, we can pass these messages to the chat model
response = chat_model.invoke(messages)

In [6]:
# The response is again an AIMessage object
print(type(response))

# And we can print the content of the response
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
Absolutely! Here's a **simple, delicious, and foolproof chocolate cake recipe** that's perfect for beginners (no fancy equipment needed) and yields a moist, rich, and flavorful cake. I’ve included pro tips to avoid common mistakes.

---

### 🍫 **Classic Chocolate Cake** (Serves 8–10)
**Total time**: ~30 mins (prep) + 25 mins (bake)  
**Yields**: 1 standard cake (≈12"x12" pan)

#### 📝 **Ingredients**
| **Item**              | **Amount**          | **Why it matters**                                  |
|------------------------|---------------------|-----------------------------------------------------|
| All-purpose flour      | 1 ½ cups (190g)     | Standard for structure (use cake flour for extra tenderness) |
| Unsalted butter        | 1 cup (226g)        | Softened, room temp (for smooth batter)              |
| Granulated sugar       | 1 cup (200g)        | Helps create moisture and crumb structure            |
| Eggs                  |

### Continuning the chat

To make a chat application that keeps memory of the previous chats, we can append the generated `AIMessage` to the list of messages that we started with and then append new `HumanMessage`s after that. While this provides the full context to the LLM, after some messages the context is going to become so large that it will not fit the LLM. Additional steps will need to be made (e.g., summarization of previous chats, among others), but those are beyond the scope of this tutorial.

In [7]:
# Append the response to the messages list
messages.append(response)

# Get a new message from the user
new_human_message = HumanMessage(content="I would like a vegan version.")

messages.append(new_human_message)

# Invoke the model with the updated messages
response = chat_model.invoke(messages)

In [8]:
print(response.content)

Absolutely! Here's a **perfectly vegan, moist, and rich chocolate cake** that’s **easy to make** (no special tools needed) and **delicious without any animal products**. I’ve optimized the recipe for *maximum moisture* and *minimal fuss* — critical for vegan cakes (which can dry out easily).

---

### 🌱 **Vegan Chocolate Cake** (Serves 8–10)  
**Total time**: ~35 mins (prep) + 25 mins (bake)  
**Yields**: 1 standard cake (≈8–9 inch round pan)  

#### 📝 **Key Vegan Substitutions** (No animal products!)
| **Original**          | **Vegan Swap**               | **Why it works**                                                                 |
|------------------------|-------------------------------|-------------------------------------------------------------------------------|
| Unsalted butter        | **1 cup (226g) coconut oil** | Neutral flavor, high fat content = **moisture** (better than plant butter)     |
| Eggs                   | **1 tbsp flax seeds + 3 tbsp water** (1 flax egg

### Invocation with dictionaries

Except from LangChain messages, dictionaries can be used. Dictionaries have to be in the OpenAI chat completions format. These offer more flexibility with specific types of data, but are not well-suited for all models. For more information, please see:

https://platform.openai.com/docs/api-reference/chat/create

In [9]:
messages = [
    {"role": "system", "content": "You are a poetry expert"},
    {"role": "user", "content": "Write a haiku about spring"},
    {"role": "assistant", "content": "Cherry blossoms bloom..."}
]
response = chat_model.invoke(messages)

In [10]:
print(response.content)

  
Soft winds carry whispers of hope,  
Spring's gentle embrace.

Wait, that's not a haiku. Let me fix it.

Cherry blossoms bloom,  
Soft winds whisper hope's soft sigh,  
Spring's gentle embrace.

Hmm, still not quite right. Let me think... A haiku has 5-7-5 syllables. Let me count:

Cherry blossoms bloom (5)  
Soft winds whisper hope's soft sigh (7)  
Spring's gentle embrace (5)

Okay, that's 5-7-5. But the second line has "hope's soft sigh" which is a bit awkward. Let me rephrase.

Cherry blossoms bloom,  
Soft winds carry hope's soft sigh,  
Spring's gentle embrace.

No, "carry hope's soft sigh" is 6 syllables. Let me try:

Cherry bloss: 3, blossoms: 2, bloom: 1. That's 6, too many. Wait, "Cherry blossoms" is 4 syllables: "Che-ry" (2) "bloss-oms" (2). So "Cherry blossoms bloom" is 5 syllables: "Che-ry" (2) "bloss-oms" (2) "bloom" (1).

Okay, let's do it right. Haiku rules: 5-7-5 syllables.

First line: 5 syllables.  
Second line: 7 syllables.  
Third line: 5 syllables.

Let me writ

## Streaming

All the above can be done using streamint, i.e., receiving each next word as it is generated by the model. In contrast to `invoke`, we now need to use the `stream` function of the model which returns an iterator.

For more information on streaming please see:

https://docs.langchain.com/oss/python/langchain/models#stream

In [11]:
# We start again with system and human messages
system_message = SystemMessage(content="You are a helpful assistant that provides cooking tips.")
human_message = HumanMessage(content="Can you suggest a recipe for chocolate cake?")

messages = [system_message, human_message]

# To keep the final response, we need to keep all the generated chunks as they stream in
full = None

for chunk in chat_model.stream(messages):
    print(chunk.text, end="|", flush=True)
    full = chunk if full is None else full + chunk

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||Absolutely|!| Here|'s| a| **|simple|,| delicious|,| and| fool|proof| chocolate| cake| recipe|**| that|'s| perfect| for| beginners| (|and| even| for| baking| disasters|!).| It|’s| moist|,| rich|,| and| has| a| classic| chocolate| flavor| without| being| too| heavy|.| Total| time|:| ~|3|5| minutes|.

|---

|###| 🎂| Classic| Chocolate| Cake| (|Makes| |1|2| cupcakes| or| |1| standard| cake|)
|**|Why| this| works|**:| Uses| cocoa| powder| (|not| chocolate| chips|!)| for| a| smooth|,| r

In [12]:
# The response is now an AIMessageChunk object
print(type(full))

# And we can print the content of the response
print(full.content_blocks)

<class 'langchain_core.messages.ai.AIMessageChunk'>
[{'type': 'text', 'text': 'Absolutely! Here\'s a **simple, delicious, and foolproof chocolate cake recipe** that\'s perfect for beginners (and even for baking disasters!). It’s moist, rich, and has a classic chocolate flavor without being too heavy. Total time: ~35 minutes.\n\n---\n\n### 🎂 Classic Chocolate Cake (Makes 12 cupcakes or 1 standard cake)\n**Why this works**: Uses cocoa powder (not chocolate chips!) for a smooth, rich flavor. No fancy tools needed. *Pro tip*: The chocolate flavor is key—this recipe avoids a "bitter" taste by using high-quality cocoa.\n\n#### 📝 Ingredients\n| **Ingredient**       | **Amount**      | **Why it matters**                                  |\n|----------------------|-----------------|----------------------------------------------------|\n| All-purpose flour    | 1 ½ cups        | Standard for structure (no gluten overload)         |\n| Unsweetened cocoa powder | ¾ cup         | *Crucial!* Adds de

In [15]:
# Thinking
# Initialize the ChatOllama model with the specified model name
model_name = 'deepseek-r1:1.5b'

# and initialize the ChatOllama instance
chat_model = ChatOllama(
    model=model_name,
    validate_model_on_init=True,
    temperature=0.7,
    reasoning=True
)

In [16]:
# We start again with system and human messages
system_message = SystemMessage(content="You are a helpful assistant that helps with travelling proposals.")
human_message = HumanMessage(content="When is a good season to visit Anchorage?")

messages = [system_message, human_message]

# To keep the final response, we need to keep all the generated chunks as they stream in
full = None

for chunk in chat_model.stream(messages):
    print(chunk.text, end="", flush=True)
    full = chunk if full is None else full + chunk

**Best Time to Visit Anchorage: A Comprehensive Guide**

1. **Weather Considerations**
   - **Summer:** The climate is mild due to its location on the Arctic, making it less extreme than coastal areas. Fall weather is milder, suitable for visiting in mid-September.
   - **Climatic Factors:** No monsoons or heavy monsoon seasons, providing a consistent and comfortable climate.

2. **Economic Factors**
   - The Anchorage economy is growing, especially benefiting from the Arctic Environment Initiative, offering tourism opportunities despite being an economic disadvantage compared to coastal areas.

3. **Attractions**
   - **Historical Sites:** Visit the First City, UNESCO site, and explore its vibrant history.
   - **Libraries:** Explore nearby libraries for research opportunities.
   - **Museums:** Stroll downtown for exhibits on cultural history and natural history.

4. **Cultural Aspects**
   - Unique attractions include fishing spots in fall and the Arctic National Park for hiking and